In [1]:
# coding: utf-8  

In [254]:
import os,sys
import chardet  
path='corpus/'
def paths(path):
    path_collection=[]
    for dirpath,dirnames,filenames in os.walk(path):
        for file in filenames:
            fullpath=os.path.join(dirpath,file)
            path_collection.append(fullpath)
    return path_collection

words = [] #文档中出现的所有词
for i,file in enumerate(paths(path)):
# for i,file in enumerate(["corpus/01060108.txt"]):
    with open(file) as somefile:
        for line in somefile:
            linestr = line.strip().decode("gbk")
            for w in linestr.split('  '):
                tmp = w.split('/')[0]
                if len(tmp)>1 and tmp[0] == '[':
                    tmp = tmp[1:]
                words.append(tmp)

In [309]:
separator = [u'。',u'？',u'！',u'：',u'；',u'?',u'!',u':',u';']

In [310]:
#bigram
sentences = [] #文档中的所有句子，加了<BOS>和<EOS>
for i,file in enumerate(paths(path)):
# for i,file in enumerate(["corpus/01060108.txt"]):
    with open(file) as somefile:
        for line in somefile:
            linestr = line.strip().decode("gbk")
            line_list = linestr.split('  ')
            sentence = [u'<BOS>']
            for w in line_list:
                tmp = w.split('/')[0]
                if len(tmp)>1 and tmp[0] == '[':
                    tmp = tmp[1:]
                if tmp in separator:
                    sentence.append(u'<EOS>')
                    sentences.append(sentence)
                    sentence = [u'<BOS>']
                else:
                    sentence.append(tmp)
            if line_list[-1] not in separator and len(sentence) > 1:
                sentence.append(u'<EOS>')
                sentences.append(sentence)
            

In [311]:
len(sentences)

50609

In [313]:
for sent in sentences[:5]:
    print ' '.join(sent)

<BOS> 《 十日谈 》 抄本 重见天日 十八 幅 插图 蔚为大观 <EOS>
<BOS> 据 新华社 罗马 １月 ９日 电 （ 记者 刘 儒庭 ） 意大利 文艺复兴 时期 著名 作家 薄伽丘 （ １３１３年 — １３７５年 ） 的 名著 《 十日谈 》 手抄本 和 其中 的 １８ 幅 插图 最近 被 发现 ， 这 对 研究 这 位 作家 和 这部 世界 文学 名著 具有 重要 意义 <EOS>
<BOS> 意大利 研究 《 十日谈 》 的 著名 专家 维·布朗卡 ９日 在 罗马 林琴 科学院 介绍 了 他 的 这 一 发现 <EOS>
<BOS> 他 说 ， 这 一 手抄本 是 在 巴黎 国家 档案馆 发现 的 ， 是 薄伽丘 ４０ 岁 之前 创作 的 ， 同 他 后来 在 １３７０年 左右 重 写 的 版本 在 语言 和 风格 上 明显 不同 <EOS>
<BOS> 薄伽丘 早年 写 的 这 一 版本 是 严格 按 写作 规范 写 的 ， 后来 的 版本 写作 风格 则 更 自由 一些 <EOS>


In [314]:
sentences[0]

[u'<BOS>',
 u'\u300a',
 u'\u5341\u65e5\u8c08',
 u'\u300b',
 u'\u6284\u672c',
 u'\u91cd\u89c1\u5929\u65e5',
 u'\u5341\u516b',
 u'\u5e45',
 u'\u63d2\u56fe',
 u'\u851a\u4e3a\u5927\u89c2',
 u'<EOS>']

In [315]:
total_sentences = len(sentences)

In [316]:
total_words = len(words)

In [317]:
total_files = len(paths(path))

In [318]:
print "文档数：",total_files
print "句子数：",total_sentences
print "词数：",total_words

文档数： 3148
句子数： 50609
词数： 1120721


In [6]:
def wordcount(words):
    # 文章字符串前期处理
    strl_ist = words
    count_dict = {}
    # 如果字典里有该单词则加1，否则添加入字典
    for str in strl_ist:
        if str in count_dict.keys():
            count_dict[str] = count_dict[str] + 1
        else:
            count_dict[str] = 1
    #按照词频从高到低排列
    count_list=sorted(count_dict.iteritems(),key=lambda x:x[1],reverse=True)
    return count_list



In [ ]:
count_list = wordcount(words)

In [9]:
f1 = open('1.txt','w')

In [16]:
with open('1.txt','w') as f1:
    for i,c in enumerate(count_list):
        f1.write((c[0]+"\t"+str(c[1])+"\n").encode('utf-8'))

In [14]:
for i,c in enumerate(count_list[:10]):
    print i,"\t",c[0],"\t",c[1]

0 	， 	74921
1 	的 	54476
2 	。 	35983
3 	、 	23116
4 	在 	12022
5 	了 	11557
6 	和 	10919
7 	是 	9819
8 	“ 	7970
9 	” 	7943


In [23]:
count_list[0]

list

In [25]:
count_dict = {}

In [26]:
for c in count_list:
    count_dict[c[0]] = c[1]

In [176]:
count_dict[u"扶贫"]

363

In [179]:
uni.unigram[u'扶贫']

363

In [18]:
with open('freq.txt','w') as f1:
    for i,c in enumerate(count_list):
        f1.write((str(i+1)+"\t"+c[0]+"\t"+str(c[1])+"\n").encode('utf-8'))

In [19]:
with open('freq.txt','w') as f1:
    for i,c in enumerate(count_list):
        f1.write((str(i+1)+"\t"+c[0]+"\t"+str(c[1])+"\n").encode('gbk'))

In [319]:
class NGram(object):

    def __init__(self, n):
        # n is the order of n-gram language model
        self.n = n
        self.unigram = {}
        self.bigram = {}
        self.uniSize = 0 #不同unigram个数
        self.biSize = 0 #不同bigram个数

    # scan a sentence, extract the ngram and update their
    # frequence.
    #
    # @param    sentence    list{str}
    # @return   none
    def scan(self, sentence):
        # file your code here
        for line in sentence:
            self.ngram(line)
            
    # caluclate the ngram of the words
    #
    # @param    words       list{str}
    # @return   none
    def ngram(self, words):
        # unigram
        if self.n == 1:
            for word in words:
                if word not in self.unigram:
                    self.unigram[word] = 1
                    self.uniSize = self.uniSize +1
                else:
                    self.unigram[word] = self.unigram[word] + 1

        # bigram
        if self.n == 2:
            pre_word = words[0]
            for w in words[1:]:
                stri = pre_word + " " + w
                if stri not in self.bigram:
                    self.bigram[stri] = 1
                    self.biSize = self.biSize +1
                else:
                    self.bigram[stri] = self.bigram[stri] + 1
                pre_word = w 

In [320]:
uni = NGram(1)
bi = NGram(2)
uni.scan(sentences)
bi.scan(sentences)

In [321]:
print uni.uniSize, bi.biSize

55411 457709


In [322]:
sentence1 = u"扶贫 开发 工作 取得 很 大 成绩"
sentence2 = u"扶贫 开发 工作 得到 很 大 成绩"
sentence1a = u"<BOS> 扶贫 开发 工作 取得 很 大 成绩"
sentence2a = u"<BOS> 扶贫 开发 工作 得到 很 大 成绩"
sentence1b = u"<BOS> 扶贫 开发 工作 取得 很 大 成绩 <EOS>"
sentence2b = u"<BOS> 扶贫 开发 工作 得到 很 大 成绩 <EOS>"

In [339]:
for w in senstence1.split(' '):
    print w,uni.unigram[w]

扶贫 363
开发 484
工作 2412
取得 582
很 912
大 2244
成绩 230


1. 第一行输出句子1的unigram句子概率，并分别输出每个unigram的概率。
2. 第二行输出句子1的bigram句子概率，并分别输出每个bigram的条件概率。
3. 第三行输出句子1a的bigram句子概率，并补充输出与<BOS>相关的bigram条件概率。
4. 第四行输出句子1b的bigram句子概率，并补充输出与<BOS>和<EOS>相关的bigram条件概率。

In [323]:
def unigram_prob(unigram):
    return float(uni.unigram[unigram])/float(total_words)

In [324]:
def unigram_sentence_prob(sentence):
    word_list = sentence.split(' ')
    if word_list[0] == u'<BOS>':
        word_list = word_list[1:]
    if word_list[-1] == u'<EOS>':
        word_list = word_list[:-1]
    uni_prob = float(1)
    for w in word_list:
        w_prob = unigram_prob(w)
        uni_prob = uni_prob * w_prob
    return uni_prob

P(<s> I want english food </s>) = P(I|<s>) ×  P(want|I) × P(english|want) ×  P(food|english)  ×  P(</s>|food) =  .000031

In [325]:
def generateBigram(sentence):
    word_list = sentence.split(' ')
    bigram_list = []
    pre_word = word_list[0]
    for w in word_list[1:]:
        bigram_list.append(pre_word + " " + w)
        pre_word = w 
    return bigram_list

In [326]:
def bigram_prob(bigram):
    bi_list = bigram.split(' ')
    pre_word = bi_list[0]
    pre_word_count = uni.unigram[pre_word]
    if bigram not in bi.bigram:
        bi.bigram[bigram] = 0
    bigram_count = bi.bigram[bigram]
    return float(bigram_count)/float(pre_word_count)

In [327]:
def bigram_sentence_prob(sentence):
    word_list = sentence.split(' ')
    bi_list = generateBigram(sentence)
    bi_sent_prob = 0
    if word_list[0] == u'<BOS>':
        bi_sent_prob = 1
    else:
        bi_sent_prob = unigram_prob(word_list[0])
    for b in bi_list:
        bi_prob = bigram_prob(b)
        bi_sent_prob = bi_sent_prob * bi_prob
    return bi_sent_prob

In [328]:
unigram_sentence_prob(sentence1b)

5.227755085591562e-23

In [329]:
unigram_sentence_prob(sentence2)

3.350434101246826e-23

In [330]:
import numpy as np

In [331]:
def formatF(number):
    return "%.6f" % np.log10(number)

In [332]:
# 行内以tab分隔，概率输出取log（10为底）结果，小数点后保留6位。
with open("3.txt","wb") as f:
    #第一行输出句子1的unigram句子概率，并分别输出每个unigram的概率。
    f.write(formatF(unigram_sentence_prob(sentence1))+"\t")
    for w in sentence1.split(' '):
        f.write(formatF(unigram_prob(w))+"\t")
    f.write("\n")
    
    #第二行输出句子1的bigram句子概率，并分别输出每个bigram的条件概率。
    f.write(formatF(bigram_sentence_prob(sentence1))+"\t")
    for b in generateBigram(sentence1):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")
    
    #第三行输出句子1a的bigram句子概率，并补充输出与<BOS>相关的bigram条件概率。
    f.write(formatF(bigram_sentence_prob(sentence1a))+"\t")
    for b in generateBigram(sentence1a):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")
    
    #第四行输出句子1b的bigram句子概率，并补充输出与<BOS>和<EOS>相关的bigram条件概率。
    f.write(formatF(bigram_sentence_prob(sentence1b))+"\t")
    for b in generateBigram(sentence1b):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")
    
    #第一行输出句子2的unigram句子概率，并分别输出每个unigram的概率。
    f.write(formatF(unigram_sentence_prob(sentence2))+"\t")
    for w in sentence2.split(' '):
        f.write(formatF(unigram_prob(w))+"\t")
    f.write("\n")
    
    #第二行输出句子2的bigram句子概率，并分别输出每个bigram的条件概率。
    f.write(formatF(bigram_sentence_prob(sentence2))+"\t")
    for b in generateBigram(sentence2):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")
    
    #第三行输出句子2a的bigram句子概率，并补充输出与<BOS>相关的bigram条件概率。
    f.write(formatF(bigram_sentence_prob(sentence2a))+"\t")
    for b in generateBigram(sentence2a):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")
    
    #第四行输出句子2b的bigram句子概率，并补充输出与<BOS>和<EOS>相关的bigram条件概率。
    f.write(formatF(bigram_sentence_prob(sentence2b))+"\t")
    for b in generateBigram(sentence2b):
        f.write(formatF(bigram_prob(b))+"\t")
    f.write("\n")

In [333]:
uni.uniSize

55411

In [334]:
bi.biSize

457709